# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# import packages and modifying output settings
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Remember to enter in the cell twice for this to work

# Loading files
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Creating Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Merging datasets into one DataFrame
school_data_complete = pd.merge(student_data, school_data, how="left", 
                                on=["school_name", "school_name"])

# Removing unnecessary columns
school_data_analysis = school_data_complete[['school_name','Student ID','grade',
                                             'reading_score','math_score',
                                             'School ID', 'type','size','budget']]
# Initial overview of DataFrame
school_data_analysis.head(3)

# Printing a list of column names for reference
list(school_data_analysis)

# stylistic note: _calc and _pre are my interim step variables 
# to make the variables I need 

['school_name',
 'Student ID',
 'grade',
 'reading_score',
 'math_score',
 'School ID',
 'type',
 'size',
 'budget']

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Total number of schools
school_total = school_data_analysis['school_name'].nunique()

In [3]:
# Total number of students
student_total = school_data_analysis['Student ID'].count()

In [4]:
# Total budget
district_total_budget = sum(school_data_analysis['budget'].unique())

In [5]:
# Average math score 
avg_math_score = round(school_data_analysis['math_score'].mean(),2)

In [6]:
# Average reading score
avg_read_score = round(school_data_analysis['reading_score'].mean(),2)

In [7]:
# Overall passing rate 
overall_pass_rate = round(((school_data_analysis['math_score'].mean()) + 
                           (school_data_analysis['reading_score'].mean())) / 2, 2)

In [8]:
# Percentage of students with a passing math score (70 or greater)
# .tolist() was useful here
math_pass_calc = round(((school_data_analysis['math_score'] >= 70)
                        .value_counts("True")) * 100, 2).tolist()
math_pass = math_pass_calc[0]

In [9]:
# Percentage of students with a passing reading score (70 or greater)
reading_pass_calc = round(((school_data_analysis['reading_score'] >= 70)
                           .value_counts("True")) * 100, 2).tolist()
reading_pass = reading_pass_calc[0]

In [18]:
# Creating dataframe with the above datapoints

# school_data_analysis["district_total_budget"] = school_data_analysis["district_total_budget"].map("${:.2f}".format)

dataSummary1 = pd.DataFrame({'School Total': school_total, 'Student Total': student_total,
                            'District Total Budget': district_total_budget,'Average Math Score':
                            avg_math_score,'Average Reading Score': avg_read_score,
                            'Overall Passing %': overall_pass_rate, 'Math Passing % (score >= 70)':
                            math_pass, 'Reading Passing % (score >= 70)': reading_pass},index=[0])

dataSummary1

,School Total,Student Total,District Total Budget,Average Math Score,Average Reading Score,Overall Passing %,Math Passing % (score >= 70),Reading Passing % (score >= 70)
0,15,39170,24649428,78.99,81.88,80.43,74.98,85.81


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [11]:
# Creating a groupby object by school_name and type
byschool_summary = school_data_analysis.groupby(['school_name','type'])

# Making a list of school names
school_names = byschool_summary.median().index.get_level_values(0).tolist()

# Making a list of school type
school_type = byschool_summary.median().index.get_level_values(1).tolist()

In [12]:
# Total number of students
byschool_student_total = round(school_data_analysis['school_name'].value_counts(),2).tolist()

# Creating an array for further calculations
byschool_student_total_calc = np.asarray(byschool_student_total)

In [13]:
# School budgets
byschool_budgets_total = school_data_analysis.groupby(['school_name']).mean()['budget'].tolist()

# Creating an array for further calculations
byschool_budgets_total_calc = np.asarray(byschool_budgets_total)

In [14]:
# Budget amount spent per student
byschool_budget_per = (byschool_budgets_total_calc / byschool_student_total_calc).tolist()

In [15]:
# Average math scores by school
byschool_math_avgs = round(school_data_analysis.groupby(['school_name']).mean()['math_score'],2).tolist()

In [16]:
# Average reading scores by school
byschool_reading_avgs = round(school_data_analysis.groupby(['school_name']).mean()['reading_score'],2).tolist()

In [30]:
# % students passing math
byschool_reading_pass_calc_pre = school_data_analysis.groupby(['school_name'],as_index=False)['math_score'].agg({'math scores':(lambda x: list(x))})
byschool_reading_pass_calc_pre
# need the appropriate conditional logic 

,school_name,list
0,Bailey High School,"[59, 58, 86, 89, 61, 85, 70, 99, 80, 62, 95, 7..."
1,Cabrera High School,"[94, 97, 83, 92, 71, 72, 97, 70, 89, 96, 99, 8..."
2,Figueroa High School,"[87, 84, 77, 64, 64, 93, 90, 75, 77, 73, 75, 6..."
3,Ford High School,"[94, 63, 59, 58, 55, 75, 59, 84, 67, 93, 84, 9..."
4,Griffin High School,"[68, 81, 89, 85, 83, 71, 91, 73, 79, 76, 80, 6..."
5,Hernandez High School,"[88, 93, 81, 91, 93, 81, 71, 93, 97, 60, 99, 8..."
6,Holden High School,"[92, 81, 93, 90, 85, 99, 69, 98, 79, 89, 86, 8..."
7,Huang High School,"[79, 61, 60, 58, 84, 94, 80, 69, 87, 84, 70, 7..."
8,Johnson High School,"[87, 62, 77, 85, 85, 84, 73, 58, 80, 56, 56, 8..."
9,Pena High School,"[75, 76, 91, 71, 92, 84, 97, 72, 88, 86, 77, 9..."


In [17]:
# # % students passing Reading
# byschool_reading_pass_calc = round(((byschool_summary['reading_score'] >= 70)
#                            .value_counts('True')) * 100, 2).tolist()
# byschool_reading_pass_calc
# # need the appropriate conditional logic 

TypeError: '>=' not supported between instances of 'SeriesGroupBy' and 'int'

In [ ]:
# Overall Passing Rate (Average of the above two)

# use as guide 
# overall_pass_rate = round(((school_data_analysis['math_score'].mean()) + 
#                            (school_data_analysis['reading_score'].mean())) / 2, 2)
# overall_pass_rate


# school_overall_passing_rate = (school_math_pass + school_reading_pass) / 2


In [ ]:
# Creating dataframe with the above datapoints
#  # % Passing Math   # % Passing Reading  # Overall Passing % (Average of Passing Math/Reading)


dataSummary2 = pd.DataFrame({'School Name': school_names, 'Type': school_type, 
                             'Total Students': byschool_student_total, 'Total Budgets':
                            byschool_budgets_total, 'Per Student Budgets': byschool_budget_per,
                             'Average Math Score': byschool_math_avgs, 'Average Reading Score':
                             byschool_reading_avgs
                            })
dataSummary2

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [ ]:
# Top5Schools = school_summary_grouped.sort
# Top5Schools

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [ ]:
# Bottom5Schools = school_summary_grouped.sort
# Bottom5Schools

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.

In [ ]:

# # not what I needed here, but cool to have
# # Student totals by School type
# byschool_type = school_data_analysis['type'].value_counts()
# byschool_type
# type(byschool_type)